# **Interactive Visual Analytics with Folium**


The launch success rate may depend on various factors, such as payload mass, orbit type, and more. It may also be influenced by the location and proximities of the launch site, which impact the initial trajectory of the rocket. Identifying the optimal location for building a launch site involves many factors, and this analysis of existing launch site locations may help uncover some of those factors.

## Objectives


This lab involves the following steps:

* Marking all launch sites on a map.
* Marking the success and failure launches for each site on the map.
* Calculating the distances between a launch site and its proximities.




Python packages:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

#### Mark all launch sites on a map


The location of each site will be added to a map using the site's latitude and longitude coordinates.

The dataset spacex_launch_geo.csv contains the augmented data, including the latitude and longitude for each launch site.

In [5]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

The coordinates for each site can now be viewed. These include the latitude and longitude for each launch site.

In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


The coordinates above are just plain numbers, which may not provide intuitive insights about the launch sites' locations. For those not familiar with geography, these numbers can be difficult to interpret. To make it easier, the locations will be visualized by pinning them on a map.

First, a Folium Map object will be created, with the initial center location set to NASA Johnson Space Center in Houston, Texas.

In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

#### A folium.Circle and folium.Marker will be created and added for each launch site on the map.

In [10]:
# Initialize the map centered at NASA Johnson Space Center
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Loop through each launch site and add a circle and marker
for index, row in launch_sites_df.iterrows():
    launch_site_name = row['Launch Site']
    launch_site_coord = [row['Lat'], row['Long']]
    
    # Add a circle at each launch site
    folium.Circle(
        location=launch_site_coord,
        radius=1000,
        color='#000000',
        fill=True,
        fill_color='#3186cc'
    ).add_child(folium.Popup(launch_site_name)).add_to(site_map)
    
    # Add a marker with the launch site name
    folium.Marker(
        location=launch_site_coord,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{launch_site_name}</b></div>'
        )
    ).add_to(site_map)

# Display the map
site_map


##### Analysis of SpaceX Launch Sites  

###### Proximity to the Equator  

The Equator is located at 0° latitude, while SpaceX launch sites are positioned at significantly higher latitudes:  


- **CCAFS SLC-40**: **28.5632° N**  
- **KSC LC-39A**: **28.5733° N**  
- **VAFB SLC-4E**: **34.6328° N**  

None of the launch sites are near the Equator. However, the Florida sites (CCAFS and KSC) are closer to the Equator compared to VAFB SLC-4E in California, which is much farther north.  

Launch sites closer to the Equator offer advantages for equatorial and geostationary orbits. The Earth's rotational speed is highest at the Equator, providing an additional velocity boost that reduces fuel consumption and launch costs.  

---

###### Proximity to the Coast  

All SpaceX launch sites are located near coastlines:  

- *CCAFS SLC-40*: Situated at *Cape Canaveral Space Force Station*, adjacent to the Atlantic Ocean.  
- *KSC LC-39A*: Located in *Merritt Island National Wildlife Refuge*, near the Atlantic coast.  
- *VAFB SLC-4E*: Positioned in *Vandenberg State Marine Reserve, California*, near the Pacific Ocean.  

Coastal locations are strategically chosen for safety, allowing launches over open water to minimize risks to populated areas in case of failures. Additionally, launching over water enables efficient *first-stage recovery*, which is critical for SpaceX’s reusability strategy.  

---

###### Key Insights  

- SpaceX launch sites are not near the Equator, but Florida locations are closer compared to Vandenberg.  
- All launch sites are positioned near coastlines to ensure safe launches and facilitate booster recovery.  
- The Florida launch sites (CCAFS & KSC) are optimal for equatorial and geostationary orbits, while Vandenberg is used for polar orbits, which require a north-south trajectory.  

The strategic placement of these launch sites enables SpaceX to conduct a wide range of missions, from satellite deployments to crewed spaceflights.  


#### Mark the success/failed launches for each site on the map


Next, the map will be enhanced by adding the launch outcomes for each site to visualize which sites have high success rates. The spacex_df dataframe contains detailed launch records, and the class column indicates whether each launch was successful or not.

In [12]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, markers will be created for all launch records. If a launch was successful (class=1), a green marker will be used, and if the launch was failed (class=0), a red marker will be used.

Since launches occur at one of four launch sites, many records will have the same coordinates. To simplify the map, MarkerCluster will be used to group markers with identical coordinates.

First, a MarkerCluster object will be created.

In [13]:
marker_cluster = MarkerCluster()


#### A new column called marker_color will be created in the spacex_df dataframe to store the marker colors based on the class value.

In [16]:
# Create the 'marker_color' column based on the 'class' column
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Now, let's add the markers to the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Iterate through the DataFrame and add markers for each launch
for index, row in spacex_df.iterrows():
    # Extract latitude, longitude, and marker color
    lat, lon, color = row['Lat'], row['Long'], row['marker_color']
    
    # Create a marker with the respective color
    folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(color=color),
        popup=f"Launch Site: {row['Launch Site']}\nOutcome: {'Success' if color == 'green' else 'Failure'}"
    ).add_to(marker_cluster)

# Display the map
site_map


##### Launch Site Success Analysis

From the map, the launch outcomes for each site reveal the following patterns:

1. *KSC LC-39A*:
   - This site has the highest number of successful launches. Out of 13 launches, 10 were successful (green markers), and 3 were failures (red markers). This indicates that KSC LC-39A has a strong success rate, making it a reliable launch site for SpaceX.

2. *CCAFS SLC-40*:
   - The CCAFS SLC-40 site has several failures compared to successes. This site shows a significantly higher failure rate, with more than 10 failed launches (red markers) and only a few successes (green markers). The number of failures at this site suggests that improvements are needed to increase the reliability of launches here.

3. *VAFB SLC-4E*:
   - VAFB SLC-4E has 4 successful launches and 6 failures. While the success rate is lower than KSC LC-39A, this site shows a balanced mix of successes and failures. The failure rate here is notable but not as high as CCAFS SLC-40.

Overall, KSC LC-39A stands out with the highest success rate, while CCAFS SLC-40 is a site with a significantly higher failure rate, suggesting that the site needs attention to improve its performance. VAFB SLC-4E, though slightly less successful, shows a more balanced outcome.


#### For each launch result in the spacex_df dataframe, a folium.Marker will be added to the marker_cluster.


In [17]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# For each row in spacex_df, create a Marker object with its coordinate and customized icon
for index, record in spacex_df.iterrows():
    lat, lon = record['Lat'], record['Long']
    color = record['marker_color']
    
    # Create a Marker with the specified color (green for success, red for failure)
    marker = folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(color=color, icon_color='white'),
        popup=f"Launch Site: {record['Launch Site']}\nOutcome: {'Success' if color == 'green' else 'Failure'}"
    )
    
    # Add the marker to the MarkerCluster
    marker_cluster.add_child(marker)

# Display the map with the added markers
site_map


#### The distances between a launch site and its proximities will be calculated.


Next, a MousePosition will be added to the map to display the coordinates when the mouse hovers over a point. This will allow easy identification of coordinates for points of interest (such as railways) while exploring the map.

In [19]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [20]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

#### A point will be marked on the closest coastline using MousePosition, and the distance between the coastline point and the launch site will be calculated.


In [21]:
# Example coordinates for a coastline point 
coastline_lat = 28.56367
coastline_lon = -80.57163

# Example launch site coordinates (e.g., for CCAFS SLC-40)
launch_site_lat = 28.563197
launch_site_lon = -80.576820

# Calculate the distance between the launch site and coastline
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Print the result
print(f"Distance to closest coastline: {distance_coastline} km")


Distance to closest coastline: 0.5097431144955059 km


In [22]:
# Create and add a folium.Marker on the closest coastline point
coastline_coordinate = [28.56367, -80.57163]  
distance_marker = folium.Marker(
    coastline_coordinate,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_coastline),
    )
)

# Add the marker to the map
site_map.add_child(distance_marker)

# Display the map
site_map


#### A PolyLine will be drawn between the launch site and the selected coastline point.


In [23]:
# Define coordinates for launch site and coastline
launch_site_coordinate = [28.573255, -80.646895]  # Replace with actual launch site coordinates
coastline_coordinate = [28.56367, -80.57163]  # Replace with your coastline coordinates

# Create a PolyLine object between the launch site and the coastline
coordinates = [launch_site_coordinate, coastline_coordinate]
lines = folium.PolyLine(locations=coordinates, weight=2, color='blue')

# Add the PolyLine to the map
site_map.add_child(lines)

# Display the map
site_map


In [26]:
# Define the coordinates for the launch site (KSC LC-39A) and the coastline
launch_site_coordinate = [28.573255, -80.646895]  # Example: KSC LC-39A
coastline_coordinate = [28.559, -80.601]  # Example: Closest coastline

# Create a PolyLine object to draw a line between the launch site and coastline
coordinates = [launch_site_coordinate, coastline_coordinate]
lines = folium.PolyLine(locations=coordinates, weight=1, color='blue')

# Add the PolyLine to the map
site_map.add_child(lines)

# Display the map with the PolyLine added
site_map


#### Similarly, a line can be drawn between a launch site and its closest city, railway, highway, etc. MousePosition will be used to find the coordinates of these locations on the map first.


In [24]:
# Use the coordinates from MousePosition for the point of interest (e.g., city, railway, etc.)
point_of_interest_coordinate = [28.559, -80.601]  # These should be from MousePosition

# Calculate the distance to the point of interest
distance_to_point = calculate_distance(launch_site_lat, launch_site_lon, point_of_interest_coordinate[0], point_of_interest_coordinate[1])

# Create a marker for the point of interest with the distance as popup text
distance_marker = folium.Marker(
    point_of_interest_coordinate,
    icon=folium.Icon(color='purple'),
    popup=f"Distance: {distance_to_point:.2f} KM"
)

# Add the marker to the map
site_map.add_child(distance_marker)

# Draw a line between the launch site and the point of interest
coordinates = [launch_site_coordinate, point_of_interest_coordinate]
line = folium.PolyLine(locations=coordinates, weight=2, color='purple')
site_map.add_child(line)

# Display the map
site_map


##### Analysis of Proximities to Railways, Highways, Coastlines, and Cities:

###### Launch Sites in Close Proximity to Railways:
- **CCAFS LC-40** is located close to a railway, specifically the NASA Railway. This proximity may be beneficial for transporting large equipment and materials to and from the launch site, which can support efficient mission operations.

###### Launch Sites in Close Proximity to Highways:
- **KSC LC-39A** is near a highway called **Kennedy Parkway North**, which could facilitate quick access to the launch site. This proximity is important for providing transport routes for materials, personnel, and potentially aiding in emergency evacuation.

###### Launch Sites in Close Proximity to Coastlines:
- Both **KSC LC-39A** and **CCAFS LC-40** are located close to coastlines. For **KSC LC-39A**, the distance to the coastline is approximately **2.41 KM**. Launching from coastal sites is strategic as it provides a clear and safe trajectory for rockets, reducing risks to populated areas. Proximity to the coastline also helps with the safe disposal of spent rocket stages.

###### Launch Sites' Distance from Cities:
- **KSC LC-39A** and **CCAFS LC-40** are somewhat distant from nearby cities, with **Tituville** city being farther away. There is a natural barrier between the launch sites and the city, often water, which helps ensure public safety during rocket launches. The distance to cities also prevents potential disruptions from population density near launch zones.

###### Conclusion:
Overall, the launch sites are strategically positioned to balance proximity to key infrastructure (railways, highways, coastlines) with maintaining a safe distance from urban areas, ensuring operational efficiency and safety.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


## Completed by: 
Qazi Fabia Hoq


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
